# BLU11 - Learning Notebook - Part 1 of 3 - Working with Large Matrices

In [1]:
import numpy as np
import scipy as sp

from scipy.sparse import random,  coo_matrix, lil_matrix, dok_matrix

# 1 Space Complexity

Huge matrices require much memory, and some large matrices are very sparse, i.e., with many more zero values than data.

This allocation is a waste of resources, as missing values and data cost the same space, but only the later hold any information.

In practice, this leads to matrices that don't fit in memory, despite having a manageable amount of data.

# 2 Time Complexity

Even when we can fit a considerable matrix into memory, it doesn't ensure we can perform operations on it.

Most operations require multiple operations on each row, column, or even the individual values of the matrix.

Vectorization, which we learned previously, is a way to deal with time complexity. 

# 3 Sparse Matrices

The premise of sparse data structures is that we store only non-zero values, and assume the rest of them are zeros.

Sparse matrices allow us to mitigate both space and time complexity:
* They are less memory-intensive, as they squeeze out the zeros and store only relevant values
* Operations ignore zero values, i.e., the majority of the cells.

## 3.1 Sparse Matrices in SciPy

The `scipy.sparse` module implements sparse matrices based in regular NumPy arrays.

For the sake of objectivity, let's compare the sizes of a sparse versus a regular matrix.

We use `sp.sparse.random` to generate a sparse matrix of a given shape and density, with randomly distributed values.

(The term density is often used to refer to what we called the sparsity score previously.)

In [2]:
def sparse_matrix_nbytes(M):
    return M.data.nbytes + M.indptr.nbytes + M.indices.nbytes


A = random(10 ** 3, 10 ** 5, density=.01, format='csr')
sparse_matrix_nbytes(A) / A.toarray().nbytes

0.015005005

So, there's that.

Let's explore how sparse matrices work and exemplify some implementations.

### 3.1.1 Dictionary of Keys (DOK)

The most straightforward implementation of a sparse matrix is as a dictionary of keys, in which the keys are tuples represent indices.

```
┌───┬───┬───┐          
│ 1 │ 0 │ 0 │          {  
├───┼───┼───┤            (0, 0): 1,
│ 0 │ 1 │ 0 │ → DoK →    (1, 1): 1,
├───┼───┼───┤            (2, 2): 1,
│ 0 │ 0 │ 1 │          }
└───┴───┴───┘ 
```

In [3]:
B = random(5, 5, density=.04, format='dok', random_state=42)

B.toarray()

array([[0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.79654299, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ]])

In [4]:
dict(B)

{(1, 1): 0.7965429868602328}

### 3.1.2 Linked List (LIL)

The row-based linked list format uses two NumPy arrays with regular Python lists inside them, one list per row.

The first array stores the values in the cells ordered from the left-most column to the right-most one. (If there's no data, the row is an empty list.)

The second stores which cells are occupied, i.e., the column index of the values.

```
┌───┬───┬───┐         Matrix data:    Matrix rows:          
│ 1 │ 0 │ 1 │          [               [  
├───┼───┼───┤            [1, 1],         [0, 2],
│ 0 │ 0 │ 0 │ → LIL →    [],             [],
├───┼───┼───┤            [1],            [2]
│ 0 │ 0 │ 1 │          ]               ]
└───┴───┴───┘ 
```

In [5]:
C = random(5, 5, density=.04, format='lil', random_state=45)

C.toarray()

array([[0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.30453644, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ]])

In [6]:
C.data

array([list([]), list([0.304536436122244]), list([]), list([]), list([])],
      dtype=object)

In [7]:
C.rows

array([list([]), list([2]), list([]), list([]), list([])], dtype=object)

### 3.1.3 Coordinate List (COO)

A `COO` matrix uses three NumPy arrays, and for each occupied cell, there is a value in the three of them.

Perhaps without surprise, the first stores the values, the second the row indexes and the third the column indexes.

```
┌───┬───┬───┐                   
│ 1 │ 0 │ 1 │          Matrix data: [1, 1, 1] 
├───┼───┼───┤          
│ 0 │ 0 │ 0 │ → COO →  Matrix rows: [0, 0, 2]
├───┼───┼───┤          
│ 0 │ 0 │ 1 │          Matrix cols: [0, 2, 2]
└───┴───┴───┘ 
```

A particularity of `COO` matrices, is that they allow duplicate entries.

In [8]:
D = random(5, 5, density=.04, format='coo', random_state=54)

D.toarray()

array([[0.        , 0.        , 0.        , 0.19393333, 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ]])

In [9]:
D.data

array([0.19393333])

In [10]:
D.row

array([0], dtype=int32)

In [11]:
D.col

array([3], dtype=int32)

### 3.1.4 Compressed Sparse (CS)

The Compressed Sparse Row (CSR), uses three arrays:
* `data`, the value vector containing all non-zero values in [row-major order](https://en.wikipedia.org/wiki/Row-_and_column-major_order)
* `indptr`, the index pointer indicates at which element of the value vector the row starts
* `indices`, contains the column indices (which column each of the values come from).

```
┌───┬───┬───┐                   
│ 1 │ 0 │ 1 │          Matrix data:    [1, 1, 1] 
├───┼───┼───┤          
│ 0 │ 0 │ 0 │ → CSR →  Matrix indptr:  [0, 2, 2, 4]
├───┼───┼───┤          
│ 0 │ 0 │ 1 │          Matrix indices: [0, 2, 2]
└───┴───┴───┘ 
```

In fact, the index pointers tell us the starting and stopping indices `data[i, j]` for each row, above:
* The first row is given by `data[0:2]`
* The second row is given by `data[2:2]`
* The third row is given by `data[2:4]`.

The Compressed Sparse Column (CSC) format is similar, but the pointers refer to columns and the indices to the rows.

Finally, by default when converting from `COO` to `CSR` or `CSC` format, duplicate entries are summed together. 

In [12]:
E = random(5, 5, density=.2, format='csr', random_state=65)

E.toarray()

array([[0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.57036289],
       [0.        , 0.11738351, 0.        , 0.        , 0.        ],
       [0.        , 0.64199587, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.41996239, 0.        , 0.95389536]])

In [13]:
E.data

array([0.57036289, 0.11738351, 0.64199587, 0.41996239, 0.95389536])

In [14]:
E.indptr

array([0, 0, 1, 2, 3, 5], dtype=int32)

In [15]:
E.indices

array([4, 1, 1, 2, 4], dtype=int32)

### 3.1.5 Summary

The main takeaways (aka the sparse matrices guide to happiness):

* `DOK`, `LIL` and `COO` are all convenient formats to construct sparse matrices
* `COO` is a fast format for constructing sparse matrices and is preferred to `DOK` and `LIL` for large matrices
* On the other hand, `DOK` and `LIL` deal efficiently with changes to the sparsity structure, suited for incremental construction
* Once we construct the matrix, we convert it to `CSR` or `CSC` fast operations, the most used for write-once-read-many tasks.

| Sparse Matrix Type       | Street Name | Good                                                                                       | Bad                                                                   |
|--------------------------|-------------|--------------------------------------------------------------------------------------------|-----------------------------------------------------------------------|
| Dictionary of Keys (DOK) | DOK         | Fast incremental construction; Flexible structure; Efficient access to individual elements | Slow iteration; Slow arithmetics; Slow slicing                        |
| List of Lists (LIL)      | LIL         | Fast incremental construction; Flexible structure                                          | Slow arithmetics; Slow column slicing; Slow matrix-vector products    |
| Coordinate List (COO)    | COO         | Fast conversion to other sparse formats; Permits duplicate entries                         | Doesn't support arithmetic operations; Doesn't support slicing        |
| Compressed Sparse (CS)   | CSR/CSC     | Changes to sparsity structure are expensive                                                | Efficient arithmetics; Efficient slicing; Fast matrix-vector products |
*Table 1: Comparison of the main sparse matrix types in SciPy.*

Finally, when comparing the two types of Compressed Sparse matrices:
* `CSR` provides efficient row slicing but slow column slicing, i.e., accessing and operating on row vectors
* `CSC` provides efficient column slicing but slow row slicing, i.e., accessing and operating on column vectors.

Let's create some sparse matrices!

## 3.2 Creating Sparse Matrices

Back to our rating matrix $R$ from the previous unit, as:

```
    ┌───┬───┬───┐                   
    │ 1 │   │ 2 │
    ├───┼───┼───┤          
R = │ 1 │ 5 │   │
    ├───┼───┼───┤          
    │   │ 2 │ 1 │
    └───┴───┴───┘ 
```

In this section, we build sparse representations of $R$.

We start from our standard array.

In [16]:
data = np.array([1, 0, 2, 1, 5, 0, 0, 2, 1]).reshape(3, 3)

### 3.2.1 DOK and LIL

The use-case for both `DOK` and `LIL` is incremental construction.

In [17]:
F = dok_matrix((3, 3))

for i in range(3):
    for j in range(3):
        F[i, j] = data[i, j]

F.toarray()

array([[1., 0., 2.],
       [1., 5., 0.],
       [0., 2., 1.]])

In [18]:
G = lil_matrix((3, 3))

G[data.nonzero()] = data[data.nonzero()]

G.toarray()

array([[1., 0., 2.],
       [1., 5., 0.],
       [0., 2., 1.]])

### 3.2.2 COO

However, the most common way to create sparse matrices (and the one we will be using) is the `COO`.

In [19]:
data  = np.array([1, 2, 1, 5, 2, 1])

col = np.array([0, 2, 0, 1, 1, 2])
row = np.array([0, 0, 1, 1, 2, 2])

H = coo_matrix((data, (row, col)), shape=(3, 3))
H.toarray()

array([[1., 0., 2.],
       [1., 5., 0.],
       [0., 2., 1.]])

## 3.3 Converting to Compressed Sparse

`COO` matrices can easily be converted to the `CSR` format, so that we can efficiently operate on them.

In [20]:
H_ = H.tocsr()
H_

<3x3 sparse matrix of type '<class 'numpy.int64'>'
	with 6 stored elements in Compressed Sparse Row format>

In [21]:
H_.data

array([1, 2, 1, 5, 2, 1], dtype=int64)

In [22]:
H_.indptr

array([0, 2, 4, 6], dtype=int32)

In [23]:
H_.indices

array([0, 2, 0, 1, 1, 2], dtype=int32)

The process is exactly the same to convert `COO` to `CSC`.

In [24]:
H.tocsc()

<3x3 sparse matrix of type '<class 'numpy.int64'>'
	with 6 stored elements in Compressed Sparse Column format>

# 4 Matrix Algebra

In the previous learning unit, we introduced vectorization and data-parallelization, i.e., performing a single operation on multiple data at once.

Vectorization decreases time complexity, grounded in $n$-dimensional arrays and the methods and functions that operate on them.

Vectorized instructions are applied to the data as a whole, avoiding cycles and, particularly, `for` loops.

A particularly relevant set of methods deliberately left out of the last unit, relates to linear algebra and matrix multiplication.

## 4.1 Notation

We write $A \in \mathbb{R}^{\space m \space \times \space n}$ a matrix $m$ rows and $n$ columns, where the entries of $A$ are real numbers.

An $n$-dimensional vector is a vector with $n$ entries, as $x \in \mathbb{R}^n$, represented as a matrix with $n$ rows and one column, i.e., a column-vector.

$$x = \begin{bmatrix} x_0 \\ x_1 \\ ... \\ x_i \end{bmatrix}$$ 

If we want to represent $x$ as a matrix with $n$ columns and a single row, i.e., a row-vector, we explicitly write the vector transposed.

$$x^T = \begin{bmatrix}x_0 & x_1 & ... & x_i \end{bmatrix}$$

We use $a_{ij}$ to represent the value of $A$ in the $i$-th row and the $j$-th column, $a_j$ for the $j$-th column and $a_i^T$ for the $i$-th row.

$$A = \begin{bmatrix}a_{00} & a_{01} & ... & a_{0n} \\ a_{10} & a_{11} & ... & a_{1n} \\ ...  & ... & ... & ...\\ a_{m0} & a_{m1} & ... & a_{mn}\end{bmatrix}$$

$$a_j = \begin{bmatrix} a_{0j} \\ a_{1j} \\ ... \\ a_{mj} \end{bmatrix}$$ 

$$a_i^T = \begin{bmatrix}a_{i0} & a_{i1} & ... & a_{in} \end{bmatrix}$$

We move to matrix multiplication, the critical matrix operation in memory-based recommender systems.

## 4.2 Matrix Multiplication

The product of two matrices $A \in \mathbb{R}^{\space m \space \times \space n}$ and $B \in \mathbb{R}^{\space m \space \times \space p}$ is the matrix $C = AB \in \mathbb{R}^{\space m \space \times \space n}$, where:

$$C_{ij} = \sum\limits_{k=1}^n A_{ik}B_{kj}$$

In short, each value in $C_{ik}$ is the sum of the product of the $i$-throw of $A$ by the $j$-th column of $B$.

For the matrix product to exist, the number of columns in $A$ must equal the number of rows in $B$.

## 4.3 Vector-vector products

This product is known as the scalar or *dot* product, from the centered dot that is used to designate this operation, as in $x \cdot y$.

Given $x^T, y \in \mathbb{R}^n$, the inner product or dot-product of the vectors is:

$$x^Ty \in R = \begin{bmatrix}x_0 & x_1 & ... & x_n\end{bmatrix} \begin{bmatrix}y_0 \\ y_1 \\ ... \\ y_n\end{bmatrix} = \sum\limits_{i=1}^n x_{i}y_{i}$$

Note that this is a particular case of matrix multiplication, with a single row and a single column.

This operation is sometimes also referred to as the sum-product, popularized by an Excel function with that name.

In [25]:
x = [0, 1, 2, 3, 4]
y = [5, 6, 7, 8, 9]

Let's implement this in vanilla Python, to build up our intuition. 

We will use `zip` to aggregate both lists element-wise.

In [26]:
list(zip(x, y))

[(0, 5), (1, 6), (2, 7), (3, 8), (4, 9)]

In [27]:
def dot_product(x, y):
    result = 0
    for i, j in zip(x, y):
        result += i*j
        
    return result


dot_product(x, y)

80

It is always the case that $x^Ty = y^Tx$.

Without surprises, NumPy provides us with a vectorized function for the dot-product of two arrays.

We transpose a matrix using the array method `transposed`, or simply `T`.

In [28]:
x_ = np.array(x).reshape(-1, 1)
y_ = np.array(y).reshape(-1, 1)

np.dot(x_.T, y_)

array([[80]])

## 4.4 Matrix-Vector products

Given a matrix $A \in \mathbb{R}^{\space m \space \times \space n}$ and a vector $x \in \mathbb{R}^{\space n}$, the product is a vector $y = Ax \in \mathbb{R}^{\space m}$. 

(Perhaps it's worth remembering that $x$ is an $n \times 1$ matrix. Hence, the shape of $y$.)

There are four equivalent ways to look at matrix-vector multiplication. Take $A$ by the rows, we write it as:

$$y = Ax = \begin{bmatrix}a_0^Tx \\ a_1^Tx \\ ... \\ a_m^Tx\end{bmatrix}$$

In other words, the $i$-th entry of $y$ is equal to the inner product of the $i$-th row of $A$ and $x$, $y_i = a_i^T x$.

However, if we write $A$ in column form:

$$y = Ax = a_0x_0 + a_1x_1 + ... + a_nx_n$$

Thus, $y$ is a linear combination of the columns of $A$, where $x$ gives the coefficients.

It's also possible to multiply on the left by a row vector. Using the columns from $A$:

$$y^T = x^TA = \begin{bmatrix}x^Ta_0 & x^Ta_1 & ... & x^Ta_n \end{bmatrix}$$

Or, alternatively, using the rows:

$$y^T = x_0a_0^T + x_1a_1^T + ... + x_na_n^T$$

We have that $y^T$ is a linear combination of the rows of $A$, where $x$ is a vector of coefficients.

In [29]:
H = np.arange(9).reshape(3, 3)
H

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [30]:
h = np.arange(3).reshape(-1, 1)
h

array([[0],
       [1],
       [2]])

In [31]:
np.dot(H, h)

array([[ 5],
       [14],
       [23]])

## 4.5 Matrix-Matrix products

Using the above, we can build our intuition of the matrix-matrix multiplication $C = AB$.

We start by representing matrix multiplication as a set of dot-products.

$$C = AB = \begin{bmatrix}a_0^Tb_0 & a_0^Tb_1 & ... & a_0^Tb_p \\ a_1^Tb_0 & a_1^Tb_1 & ... & a_1^Tb_p \\ ...  & ... & ... & ...\\ a_m^Tb_0 & a_m^Tb_1 & ... & a_m^Tb_p\end{bmatrix}$$

Remember that since $A \in \mathbb{R}^{\space m \space \times \space n}$ and $B \in \mathbb{R}^{\space m \space \times \space p}$, we have $a_i \in \mathbb{R}^n$ and $b_j \in \mathbb{R}^n$.

A second approach is to represent matrix multiplication as a set of matrix-vector products, each returning a column-vector.

$$C = AB = \begin{bmatrix}Ab_0 & Ab_1 & ... & Ab_p\end{bmatrix}$$

A third one is to use matrix-vector products with the vector on the left, each returning a row-vector.

$$C = AB = \begin{bmatrix}a_0^TB \\ a_1^TB \\ ... \\ a_m^TB\end{bmatrix}$$

In [32]:
I = np.arange(12).reshape(3, 4)
I

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [33]:
np.dot(H, I)

array([[ 20,  23,  26,  29],
       [ 56,  68,  80,  92],
       [ 92, 113, 134, 155]])

Combining sparse matrices with linear algebra, we manage space and time complexity when building memory-based RS.